<a href="https://colab.research.google.com/github/GaaraPrograming/Desafio/blob/main/DesafioPhyton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DynamoFlow: Gestor de Registros Contextual en Python

In [6]:
# DynamoFlow: Gestor de Registros Contextual

from typing import Callable, Dict, List
import re
from datetime import datetime

# Definición de la clase DynamoFlow

class DynamoFlow:
    """
    DynamoFlow: gestor de registros contextuales.
    Para cada '__type__' de registro, aplica validaciones y transformaciones registradas.
    """

    def __init__(self):
        # Diccionarios que asocian cada tipo a sus listas de funciones
        self._validations: Dict[str, List[Callable[[dict], None]]] = {}
        self._transformations: Dict[str, List[Callable[[dict], None]]] = {}

        # (Opcional) Validaciones/transformaciones globales
        self._global_validations: List[Callable[[dict], None]] = []
        self._global_transformations: List[Callable[[dict], None]] = []


    # Decoradores para registrar validaciones
    def register_validation(self, tipo: str):
        """
        Decorador para registrar una función de validación bajo un tipo dado.
        La función debe recibir el registro (dict) y, en caso de falla, lanzar ValueError.
        """
        def decorator(func: Callable[[dict], None]):
            if tipo not in self._validations:
                self._validations[tipo] = []
            self._validations[tipo].append(func)
            return func
        return decorator

    # Decoradores para registrar transformaciones
    def register_transformation(self, tipo: str):
        """
        Decorador para registrar una función de transformación bajo un tipo dado.
        La función recibe el registro (dict), lo modifica “in place” o reemplaza campos.
        """
        def decorator(func: Callable[[dict], None]):
            if tipo not in self._transformations:
                self._transformations[tipo] = []
            self._transformations[tipo].append(func)
            return func
        return decorator


    # (Opcional) Decoradores para registrar validaciones/transformaciones globales
    def register_global_validation(self, func: Callable[[dict], None]):
        """
        Validador que se aplica a todos los registros, independientemente de su tipo.
        """
        self._global_validations.append(func)
        return func

    def register_global_transformation(self, func: Callable[[dict], None]):
        """
        Transformación que se aplica a todos los registros, independientemente de su tipo.
        """
        self._global_transformations.append(func)
        return func


    # Método principal: procesar un registro
    def process(self, record: dict) -> dict:
        """
        Procesa un único registro:
          1. Extrae '__type__' (o lanza ValueError si no existe).
          2. Aplica validaciones globales.
          3. Aplica validaciones específicas de tipo.
          4. Aplica transformaciones globales.
          5. Aplica transformaciones específicas de tipo.
        Devuelve el mismo diccionario (posiblemente modificado) o lanza ValueError si falla validación.
        """
        # 1. Obtener tipo
        if "__type__" not in record:
            raise ValueError("El registro no contiene el campo '__type__'. Imposible procesar.")
        tipo = record["__type__"]

        # 2. Validaciones globales
        for val_func in self._global_validations:
            val_func(record)  # debe lanzar ValueError si falla

        # 3. Validaciones específicas del tipo
        if tipo in self._validations:
            for val_func in self._validations[tipo]:
                val_func(record)

        # 4. Transformaciones globales
        for trans_func in self._global_transformations:
            trans_func(record)

        # 5. Transformaciones específicas del tipo
        if tipo in self._transformations:
            for trans_func in self._transformations[tipo]:
                trans_func(record)

        # 6. Devolver el registro (modificado “in place”)
        return record



# Instancia de DynamoFlow
dyna = DynamoFlow()


# VALIDACIONES para "order_event"
@dyna.register_validation("order_event")
def validar_campos_orden(record: dict):
    """
    Verifica que existan los campos obligatorios para un 'order_event'.
    """
    required_fields = ["order_id", "customer_name", "amount", "timestamp"]
    missing = [f for f in required_fields if f not in record]
    if missing:
        raise ValueError(f"[order_event] Faltan campos obligatorios: {missing}")

@dyna.register_validation("order_event")
def validar_amount_orden(record: dict):
    """
    Comprueba que 'amount' pueda ser float >= 0.0.
    """
    amount = record.get("amount")
    try:
        amt = float(amount)
    except Exception:
        raise ValueError(f"[order_event] El campo 'amount' ('{amount}') no es un número válido.")
    if amt < 0.0:
        raise ValueError(f"[order_event] El campo 'amount' ('{amount}') no puede ser negativo.")

@dyna.register_validation("order_event")
def validar_timestamp_orden(record: dict):
    """
    Verifica que 'timestamp' tenga formato ISO 8601 (mínimo: YYYY-MM-DDThh:mm:ssZ).
    """
    ts = record.get("timestamp", "")
    # Expresión regular muy básica para ISO 8601 con Z al final: 2023-10-26T14:00:00Z
    iso_regex = r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$"
    if not re.match(iso_regex, ts):
        raise ValueError(f"[order_event] El campo 'timestamp' ('{ts}') no cumple formato ISO 8601 (ej. 2023-10-26T14:00:00Z).")


# TRANSFORMACIONES para "order_event"
@dyna.register_transformation("order_event")
def transformar_amount_y_timestamp(record: dict):
    """
    Convierte 'amount' a float y 'timestamp' a datetime, y añade 'total_with_tax'.
    """
    # Convertir amount a float
    amount_str = record["amount"]
    amt = float(amount_str)
    record["amount"] = amt  # Sobre-escribimos con float

    # Convertir timestamp a datetime
    ts_str = record["timestamp"]
    dt_obj = datetime.strptime(ts_str, "%Y-%m-%dT%H:%M:%SZ")
    record["timestamp"] = dt_obj

    # Añadir total con impuesto (ej. 19% IVA)
    record["total_with_tax"] = round(amt * 1.19, 2)

@dyna.register_transformation("order_event")
def capitalizar_nombre_cliente(record: dict):
    """
    Capitaliza cada palabra en 'customer_name'.
    """
    cname = record["customer_name"]
    record["customer_name"] = cname.title()



# VALIDACIONES para "product_update"
@dyna.register_validation("product_update")
def validar_campos_producto(record: dict):
    """
    Verifica que existan los campos 'sku' y 'price' para un 'product_update'.
    """
    required_fields = ["sku", "price"]
    missing = [f for f in required_fields if f not in record]
    if missing:
        raise ValueError(f"[product_update] Faltan campos obligatorios: {missing}")

@dyna.register_validation("product_update")
def validar_formato_price(record: dict):
    """
    Verifica que 'price' tenga el formato '<número> <moneda>' (ej. '99.99 EUR').
    """
    precio = record.get("price", "")
    # Regex: uno o más dígitos con opcional decimal, espacio, 3 letras mayúsculas
    price_regex = r"^[0-9]+(?:\.[0-9]{1,2})?\s+[A-Z]{3}$"
    if not re.match(price_regex, precio):
        raise ValueError(f"[product_update] El campo 'price' ('{precio}') no cumple formato 'NNN.NN XXX'.")



# TRANSFORMACIONES para "product_update"
@dyna.register_transformation("product_update")
def separar_price_en_campo_y_moneda(record: dict):
    """
    Separa 'price' en 'price_value' (float) y 'currency' (str), y elimina 'price'.
    """
    precio = record["price"]
    partes = precio.split()
    valor = float(partes[0])
    moneda = partes[1]
    record["price_value"] = valor
    record["currency"] = moneda
    del record["price"]

@dyna.register_transformation("product_update")
def asegurar_is_active(record: dict):
    """
    Si no existe 'is_active', lo crea con True. Si existe, lo convierte a booleano.
    """
    if "is_active" not in record:
        record["is_active"] = True
    else:
        val = record["is_active"]
        if isinstance(val, str):
            record["is_active"] = val.strip().lower() == "true"
        else:
            record["is_active"] = bool(val)



# Ejemplos de uso / Pruebas

if __name__ == "__main__":
    # 1. Prueba: order_event válido
    record_example_1 = {
        "__type__": "order_event",
        "order_id": "ORD789",
        "customer_name": "luis vargas",
        "amount": "123.45",
        "timestamp": "2023-10-26T14:00:00Z"
    }

    try:
        processed_1 = dyna.process(record_example_1.copy())
        print("order_event procesado correctamente:")
        for k, v in processed_1.items():
            print(f"  {k}: {v}")
    except ValueError as e:
        print("Error al procesar order_event:", e)

    print("\n" + "-" * 50 + "\n")

    # 2. Prueba: product_update válido
    record_example_2 = {
        "__type__": "product_update",
        "sku": "SKU_P01",
        "price": "59.99 EUR"
        # no viene is_active => se debe agregar True
    }

    try:
        processed_2 = dyna.process(record_example_2.copy())
        print("product_update procesado correctamente:")
        for k, v in processed_2.items():
            print(f"  {k}: {v}")
    except ValueError as e:
        print("Error al procesar product_update:", e)

    print("\n" + "-" * 50 + "\n")

    # 3. Prueba: order_event inválido
    record_bad = {
        "__type__": "order_event",
        "order_id": "ORD999",
        # falta customer_name
        "amount": "-5.00",      # negativo
        "timestamp": "2023/10/26 14:00:00"  # formato incorrecto
    }

    try:
        processed_bad = dyna.process(record_bad.copy())
        print("¡Esto no debería verse, porque falla validación!")
        print(processed_bad)
    except ValueError as e:
        print("Se capturó el error esperado al procesar registro inválido:\n ", e)


order_event procesado correctamente:
  __type__: order_event
  order_id: ORD789
  customer_name: Luis Vargas
  amount: 123.45
  timestamp: 2023-10-26 14:00:00
  total_with_tax: 146.91

--------------------------------------------------

product_update procesado correctamente:
  __type__: product_update
  sku: SKU_P01
  price_value: 59.99
  currency: EUR
  is_active: True

--------------------------------------------------

Se capturó el error esperado al procesar registro inválido:
  [order_event] Faltan campos obligatorios: ['customer_name']


# DynamoFlow con Operaciones Abstractas (Operation)

In [7]:

# DynamoFlow con Operaciones Abstractas (Operation)

import abc
import re
import logging
from datetime import datetime
from typing import Dict, List

# 1. Configuración básica de logging (opcional)

# Configuramos logging para que las advertencias se vean en pantalla.
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s"
)



# 2. Clase base abstracta: Operation
class Operation(abc.ABC):
    """
    Clase base abstracta para todas las Operaciones (tanto validaciones
    como transformaciones). Cada operación recibe un 'field_name' y
    puede definirse con parámetros adicionales según su propósito.
    """

    def __init__(self, field_name: str):
        """
        :param field_name: nombre del campo del registro sobre el que
                           esta operación actuará (validar o transformar).
        """
        self.field_name = field_name

    @abc.abstractmethod
    def apply(self, record: dict) -> None:
        """
        Método que debe implementar toda subclase. Recibe el registro (un dict)
        y:
          - Si es una validación: lanza ValueError si falla la validación.
          - Si es una transformación: modifica 'record' en su lugar (in-place).
        """
        pass



# 3. Implementación de NormalizeAmountOperation
#    (Operación de Transformación)

class NormalizeAmountOperation(Operation):
    """
    Operation → Transformación de campos monetarios
    Convierte un campo que contiene un valor monetario en formato texto
    (e.g. "1.234,56 €", "$1,234.56", "1234.56USD") a un float estándar.
    - Maneja separadores decimales tanto coma como punto.
    - Elimina símbolos de moneda y caracteres no numéricos.
    - Si el campo no existe o la conversión falla, registra una WARNING y continúa.
    """

    def __init__(self, field_name: str):
        super().__init__(field_name)
        self.logger = logging.getLogger(self.__class__.__name__)

    def apply(self, record: dict) -> None:
        # 1) Verificar si el campo existe en el registro
        if self.field_name not in record:
            self.logger.warning(
                f"[NormalizeAmount] El campo '{self.field_name}' no existe en el registro; se omite normalización."
            )
            return

        raw_value = record[self.field_name]
        raw_str = str(raw_value)

        # 2) Eliminar todos los caracteres que no sean dígitos, coma, punto o signo negativo
        #    Esto quita símbolos de moneda, espacios, letras, etc.
        cleaned = re.sub(r"[^\d\.,\-]", "", raw_str)

        # 3) Determinar uso de coma y punto:
        #    - Si aparecen ambos: interpretamos que el punto es separador de miles y la coma separador decimal.
        #      → quitamos todos los puntos y reemplazamos la coma por punto.
        #    - Si aparece solo coma: la reemplazamos por punto (coma decimal).
        #    - En cualquier otro caso (solo puntos, o ninguno):
        #      dejamos los puntos como separadores decimales.
        if "," in cleaned:
            if "." in cleaned:
                # Ejemplo: "1.234,56" → "1234,56" → "1234.56"
                without_dots = cleaned.replace(".", "")
                normalized = without_dots.replace(",", ".")
            else:
                # Ejemplo: "1234,56" → "1234.56"
                normalized = cleaned.replace(",", ".")
        else:
            # No hay coma, solo punto o solo números: se deja tal cual
            normalized = cleaned

        # 4) Intentar convertir a float
        try:
            amount_float = float(normalized)
            record[self.field_name] = amount_float
        except Exception:
            self.logger.warning(
                f"[NormalizeAmount] No se pudo convertir '{raw_str}' a float para el campo '{self.field_name}'. "
                f"Cadena parseada: '{normalized}'."
            )


# 4. Implementación de ContextualFieldValidation
#    (Operación de Validación)
class ContextualFieldValidation(Operation):
    """
    Operation → Validación de campo contextual
    Asegura que un campo específico ('field_name') exista y cumpla con
    un patrón regular dado. Si falla, lanza ValueError para detener el procesamiento.
    """

    def __init__(self, field_name: str, pattern: str):
        """
        :param field_name: nombre del campo que se validará
        :param pattern: string con la expresión regular (en formato Python) que debe cumplir el valor
                        Ejemplo: r"^\d+(\.\d{2})?$" para montos "123.45" o r".+" para no vacío.
        """
        super().__init__(field_name)
        self.pattern = re.compile(pattern)

    def apply(self, record: dict) -> None:
        # 1) Verificar existencia del campo
        if self.field_name not in record:
            raise ValueError(
                f"[ContextualFieldValidation] El campo requerido '{self.field_name}' no está presente."
            )

        raw_value = record[self.field_name]
        raw_str = str(raw_value)

        # 2) Verificar que el valor cumpla el patrón regex
        if not self.pattern.match(raw_str):
            raise ValueError(
                f"[ContextualFieldValidation] El campo '{self.field_name}' ('{raw_str}') "
                f"no cumple el patrón /{self.pattern.pattern}/."
            )



# 5. Refactorización de la clase DynamoFlow
#    para usar Operation objects
class DynamoFlow:
    """
    Gestor de registros contextuales. Para cada '__type__' de registro,
    aplica en orden:
      1. Validaciones globales
      2. Validaciones específicas de tipo (una lista de Operation de validación)
      3. Transformaciones globales
      4. Transformaciones específicas de tipo (una lista de Operation de transformación)

    Las Operaciones (subclases de Operation) pueden ser configuradas
    con parámetros (p.ej. field_name y patrones de regex).
    """

    def __init__(self):
        # Para cada tipo (string), guardamos la lista de Operation de validación
        self._validations: Dict[str, List[Operation]] = {}
        # Para cada tipo, guardamos la lista de Operation de transformación
        self._transformations: Dict[str, List[Operation]] = {}

        # (Opcional) Operaciones globales, aplicables a todos los tipos
        self._global_validations: List[Operation] = []
        self._global_transformations: List[Operation] = []


    # Métodos para agregar Operaciones
    def add_validation(self, tipo: str, operation: Operation) -> None:
        """
        Añade una operación de validación al tipo de registro dado.
        :param tipo: string con el valor de '__type__' que identifica al registro.
        :param operation: instancia de subclase de Operation (ej. ContextualFieldValidation).
        """
        if tipo not in self._validations:
            self._validations[tipo] = []
        self._validations[tipo].append(operation)

    def add_transformation(self, tipo: str, operation: Operation) -> None:
        """
        Añade una operación de transformación al tipo de registro dado.
        :param tipo: string con el valor de '__type__' que identifica al registro.
        :param operation: instancia de subclase de Operation (ej. NormalizeAmountOperation).
        """
        if tipo not in self._transformations:
            self._transformations[tipo] = []
        self._transformations[tipo].append(operation)

    def add_global_validation(self, operation: Operation) -> None:
        """
        Añade una operación de validación que se aplicará a TODOS los registros,
        sin importar su '__type__'.
        """
        self._global_validations.append(operation)

    def add_global_transformation(self, operation: Operation) -> None:
        """
        Añade una operación de transformación que se aplicará a TODOS los registros.
        """
        self._global_transformations.append(operation)


    # Método principal para procesar un registro
    def process(self, record: dict) -> dict:
        """
        Procesa un único registro en este orden:
          1. Verifica que exista '__type__' en el registro, si no: ValueError.
          2. Aplica todas las validaciones globales (Operation.apply → arrojan ValueError si fallan).
          3. Aplica todas las validaciones específicas del tipo.
          4. Aplica todas las transformaciones globales.
          5. Aplica todas las transformaciones específicas del tipo.
        Devuelve el mismo dict (in-place modificado) o lanza ValueError si falla alguna validación.
        """
        # 1) Comprobar campo '__type__'
        if "__type__" not in record:
            raise ValueError("[DynamoFlow] El registro no contiene el campo '__type__'.")
        tipo = record["__type__"]

        # 2) Validaciones globales
        for op in self._global_validations:
            op.apply(record)

        # 3) Validaciones específicas de 'tipo'
        for op in self._validations.get(tipo, []):
            op.apply(record)

        # 4) Transformaciones globales
        for op in self._global_transformations:
            op.apply(record)

        # 5) Transformaciones específicas de 'tipo'
        for op in self._transformations.get(tipo, []):
            op.apply(record)

        # 6) Devolver el registro modificado (in-place)
        return record



# 6. Ejemplo de configuración y uso
if __name__ == "__main__":
    # 6.1. Crear la instancia de DynamoFlow
    dyna = DynamoFlow()


    # 6.2. Configurar OPERACIONES para el tipo "order_event"
    # a) Validaciones:
    #    - order_id debe existir y no estar vacío
    dyna.add_validation(
        "order_event",
        ContextualFieldValidation(field_name="order_id", pattern=r".+")
    )
    #    - customer_name debe existir y no estar vacío
    dyna.add_validation(
        "order_event",
        ContextualFieldValidation(field_name="customer_name", pattern=r".+")
    )
    #    - amount debe existir y tener formato numérico (acepta coma o punto)
    dyna.add_validation(
        "order_event",
        ContextualFieldValidation(field_name="amount", pattern=r"^[0-9]+(?:[.,][0-9]{1,2})?$")
    )
    #    - timestamp debe existir y tener formato ISO 8601 (YYYY-MM-DDThh:mm:ssZ)
    dyna.add_validation(
        "order_event",
        ContextualFieldValidation(field_name="timestamp", pattern=r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")
    )

    # b) Transformaciones:
    #    - NormalizeAmountOperation convierte 'amount' de string a float
    dyna.add_transformation(
        "order_event",
        NormalizeAmountOperation(field_name="amount")
    )



    # 6.3. Configurar OPERACIONES para el tipo "product_update"
    # a) Validaciones:
    #    - sku debe existir y no estar vacío
    dyna.add_validation(
        "product_update",
        ContextualFieldValidation(field_name="sku", pattern=r".+")
    )
    #    - price debe existir y tener formato monetario (acepta dígitos + coma/punto + 3 letras de moneda)
    dyna.add_validation(
        "product_update",
        ContextualFieldValidation(field_name="price", pattern=r"^[0-9]+(?:[.,][0-9]{1,2})?\s*[A-Z]{3}$")
    )

    # b) Transformaciones:
    #    - NormalizeAmountOperation convierte 'price' de string a float (quita símbolo de moneda)
    dyna.add_transformation(
        "product_update",
        NormalizeAmountOperation(field_name="price")
    )
    #    (Una vez convertido a float, ya no sabremos la moneda.
    #     Si quisiéramos separar currency, habría que implementar otra Operation específica.)


    # 6.4. Ejemplos de registros y prueba de procesamiento
    # Ejemplo 1: order_event válido
    record_order = {
        "__type__": "order_event",
        "order_id": "ORD123",
        "customer_name": "ana pérez",
        "amount": "1.234,56 €",        # Ejemplo con punto y coma como separadores
        "timestamp": "2023-10-26T14:00:00Z"
    }

    print("\n=== Procesando order_event válido ===")
    try:
        processed_order = dyna.process(record_order.copy())
        print("Registro procesado correctamente:")
        for key, val in processed_order.items():
            print(f"  {key!r}: {val!r}")
    except ValueError as e:
        print("Error al procesar order_event:", e)

    print("\n" + "-"*60 + "\n")

    # Ejemplo 2: product_update válido
    record_product = {
        "__type__": "product_update",
        "sku": "SKU_ABC",
        "price": "99.99 USD"
    }

    print("\n=== Procesando product_update válido ===")
    try:
        processed_prod = dyna.process(record_product.copy())
        print("Registro procesado correctamente:")
        for key, val in processed_prod.items():
            print(f"  {key!r}: {val!r}")
    except ValueError as e:
        print("Error al procesar product_update:", e)

    print("\n" + "-"*60 + "\n")

    # Ejemplo 3: order_event inválido (falta campo y formato de timestamp incorrecto)
    record_order_bad = {
        "__type__": "order_event",
        "order_id": "ORD999",
        # falta customer_name
        "amount": "-5,00 €",       # formato correcto pero valor negativo (aunque esto Operation no lo chequea)
        "timestamp": "26-10-2023 14:00:00"  # formato no ISO 8601
    }

    print("\n=== Procesando order_event inválido ===")
    try:
        processed_bad = dyna.process(record_order_bad.copy())
        print("¡Este mensaje no debería aparecer, porque hay validación fallida!")
        print(processed_bad)
    except ValueError as e:
        print("Se capturó el error esperado al procesar registro inválido:\n ", e)

    print("\n" + "-"*60 + "\n")

    # Ejemplo 4: product_update inválido (price en formato erróneo)
    record_product_bad = {
        "__type__": "product_update",
        "sku": "SKU_XYZ",
        "price": "XYZ99.99"  # no cumple patrón "<número> <MONEDA>"
    }

    print("\n=== Procesando product_update inválido ===")
    try:
        processed_prod_bad = dyna.process(record_product_bad.copy())
        print("¡Este mensaje no debería aparecer, porque hay validación fallida!")
        print(processed_prod_bad)
    except ValueError as e:
        print("Se capturó el error esperado al procesar registro inválido:\n ", e)



=== Procesando order_event válido ===
Error al procesar order_event: [ContextualFieldValidation] El campo 'amount' ('1.234,56 €') no cumple el patrón /^[0-9]+(?:[.,][0-9]{1,2})?$/.

------------------------------------------------------------


=== Procesando product_update válido ===
Registro procesado correctamente:
  '__type__': 'product_update'
  'sku': 'SKU_ABC'
  'price': 99.99

------------------------------------------------------------


=== Procesando order_event inválido ===
Se capturó el error esperado al procesar registro inválido:
  [ContextualFieldValidation] El campo requerido 'customer_name' no está presente.

------------------------------------------------------------


=== Procesando product_update inválido ===
Se capturó el error esperado al procesar registro inválido:
  [ContextualFieldValidation] El campo 'price' ('XYZ99.99') no cumple el patrón /^[0-9]+(?:[.,][0-9]{1,2})?\s*[A-Z]{3}$/.


# RecordContextManager: Gestor de Registros Contextual

In [8]:
# RecordContextManager: Gestor de Registros Contextual en Python

import abc
import re
import logging
from datetime import datetime
from typing import Dict, Iterable, List, Tuple, Any

# 1. Configuración básica de logging (opcional)
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s"
)


# 2. Clase base abstracta: Operation
class Operation(abc.ABC):
    """
    Clase base abstracta para todas las Operaciones (validaciones o transformaciones).
    Cada operación actúa sobre un campo específico (field_name) y puede configurarse
    con parámetros adicionales según su propósito.
    """

    def __init__(self, field_name: str):
        """
        :param field_name: nombre del campo del registro sobre el que actúa esta operación.
        """
        self.field_name = field_name

    @abc.abstractmethod
    def apply(self, record: dict) -> None:
        """
        Método que debe implementar toda subclase. Recibe el registro (dict).
        - Si es una validación: lanza ValueError si falla.
        - Si es una transformación: modifica 'record' in-place (y, opcionalmente, lanza ValueError en caso falla).
        """
        pass



# 3. Operación de Transformación: NormalizeAmountOperation
class NormalizeAmountOperation(Operation):
    """
    Transformación de campos monetarios:
    - Convierte strings con separadores de miles/puntos/comas y símbolos de moneda
      a un float estándar.
    - Si el campo no existe o la conversión falla, registra una WARNING y continúa.
    """

    def __init__(self, field_name: str):
        super().__init__(field_name)
        self.logger = logging.getLogger(self.__class__.__name__)

    def apply(self, record: dict) -> None:
        # 1) Verificar existencia del campo
        if self.field_name not in record:
            self.logger.warning(
                f"[NormalizeAmountOperation] El campo '{self.field_name}' no existe; se omite."
            )
            return

        raw_value = record[self.field_name]
        raw_str = str(raw_value)

        # 2) Limpiar la cadena: eliminar todo menos dígitos, comas, puntos y signo menos
        cleaned = re.sub(r"[^\d\.,\-]", "", raw_str)

        # 3) Normalizar separadores:
        #    - Si hay coma y punto: asumimos que punto = miles, coma = decimal.
        #    - Si hay solo coma: coma = decimal.
        #    - Si no hay coma: dejamos puntos como decimales.
        if "," in cleaned:
            if "." in cleaned:
                # Ejemplo: "1.234,56" → "1234,56" → "1234.56"
                without_dots = cleaned.replace(".", "")
                normalized = without_dots.replace(",", ".")
            else:
                # Ejemplo: "1234,56" → "1234.56"
                normalized = cleaned.replace(",", ".")
        else:
            # "1234.56" o "1234" → lo dejamos tal cual
            normalized = cleaned

        # 4) Intentar convertir a float
        try:
            amount_float = float(normalized)
            record[self.field_name] = amount_float
        except Exception:
            self.logger.warning(
                f"[NormalizeAmountOperation] No se pudo convertir '{raw_str}' → '{normalized}' a float."
            )



# 4. Operación de Validación: ContextualFieldValidation
class ContextualFieldValidation(Operation):
    """
    Validación de campo contextual:
    - Asegura que un campo (field_name) exista y cumpla un patrón regex.
    - Si no lo cumple, lanza ValueError.
    """

    def __init__(self, field_name: str, pattern: str):
        """
        :param field_name: campo que se validará.
        :param pattern: regex (string) que debe cumplir el valor del campo.
        """
        super().__init__(field_name)
        self.pattern = re.compile(pattern)

    def apply(self, record: dict) -> None:
        # 1) Verificar existencia del campo
        if self.field_name not in record:
            raise ValueError(
                f"[ContextualFieldValidation] Falta campo requerido '{self.field_name}'."
            )

        raw_value = record[self.field_name]
        raw_str = str(raw_value)

        # 2) Verificar patrón
        if not self.pattern.fullmatch(raw_str):
            raise ValueError(
                f"[ContextualFieldValidation] El campo '{self.field_name}' (valor='{raw_str}') "
                f"no coincide con el patrón /{self.pattern.pattern}/."
            )


# 5. Clase principal: RecordContextManager
class RecordContextManager:
    """
    Gestor de Contextos de Registro:
      - Permite registrar, para cada tipo de registro ('__type__'),
        una secuencia de objetos Operation (validaciones y transformaciones).
      - Procesa un flujo de registros (iterable) y, para cada registro,
        aplica en orden sus operaciones, recopilando errores y advertencias.
      - Nunca descarta registros: devuelve siempre (registro_enriquecido, lista_de_problemas).
    """

    def __init__(self):
        # Para cada tipo (string), lista ordenada de operaciones (Operation) a aplicar
        self._contexts: Dict[str, List[Operation]] = {}

    def register_context(self, tipo: str, operations: List[Operation]) -> None:
        """
        Registra (o reemplaza) la secuencia de operaciones para un tipo de registro.
        :param tipo: valor de '__type__' que identifica este contexto.
        :param operations: lista de instancias Operation (validaciones y transformaciones),
                           en el orden en que deben aplicarse.
        """
        self._contexts[tipo] = operations

    def add_operation(self, tipo: str, operation: Operation) -> None:
        """
        Agrega una operación al final de la secuencia para el tipo dado.
        Si el tipo no existía, crea la lista automáticamente.
        """
        if tipo not in self._contexts:
            self._contexts[tipo] = []
        self._contexts[tipo].append(operation)

    def process_stream(
        self,
        record_iterator: Iterable[Dict[str, Any]]
    ) -> Iterable[Tuple[Dict[str, Any], List[str]]]:
        """
        Procesa un iterable de registros (dicts). Para cada registro:
          1. Detecta su tipo '__type__' (o lo marca como error si falta).
          2. Busca la lista de Operation asociadas a ese tipo (si no existe, no hace nada).
          3. Aplica cada Operation en orden, capturando ValueError como error.
             - Si una operación lanza ValueError, se sigue con la siguiente operación,
               añadiendo el mensaje de error a la lista de problemas.
             - Las transformaciones que registran WARNINGs (p.ej. NormalizeAmountOperation)
               aparecerán en consola; no se capturan aquí.
          4. Enriquece el diccionario con:
             - "_valid": True si no hubo errores, False en caso contrario.
             - "_errors": lista de mensajes de errores (vacía si no hubo).
          5. Devuelve (yield) la tupla (registro_enriquecido, errores).
        """
        for original_record in record_iterator:
            # Hacemos una copia superficial para no mutar el original fuera de este contexto
            record = original_record.copy()
            errors: List[str] = []

            # 1) Verificar existencia de '__type__'
            tipo = record.get("__type__")
            if tipo is None:
                errors.append("Falta campo '__type__'; no se puede aplicar Contexto.")
                # No sabemos qué operaciones aplicar; enriquecemos y devolvemos
                record["_valid"] = False
                record["_errors"] = errors
                yield record, errors
                continue

            # 2) Obtener las operaciones configuradas (si existen)
            ops = self._contexts.get(tipo, [])

            # 3) Aplicar cada Operation en orden
            for op in ops:
                try:
                    op.apply(record)
                except ValueError as ve:
                    # Capturamos el mensaje de error, pero seguimos con la siguiente operación
                    errors.append(str(ve))

            # 4) Enriquecer registro con estado y errores
            record["_valid"] = (len(errors) == 0)
            record["_errors"] = errors

            # 5) Devolver la tupla (registro, lista de errores)
            yield record, errors



# 6. Ejemplo de configuración y uso
if __name__ == "__main__":
    # 6.1 Crear instancia de RecordContextManager
    manager = RecordContextManager()

    # 6.2 Configuración del contexto para "order_event"
    # - Primero: NormalizeAmountOperation para 'amount'
    # - Luego: ContextualFieldValidation para 'order_id' y 'customer_name'
    # - Finalmente: ContextualFieldValidation para 'timestamp'
    manager.register_context(
        "order_event",
        [
            NormalizeAmountOperation(field_name="amount"),
            ContextualFieldValidation(field_name="order_id", pattern=r".+"),
            ContextualFieldValidation(field_name="customer_name", pattern=r".+"),
            ContextualFieldValidation(
                field_name="timestamp",
                pattern=r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$"
            ),
        ]
    )


    # 6.3 Configuración del contexto para "product_update"
    # - Primero: NormalizeAmountOperation para 'price'
    # - Luego: ContextualFieldValidation para 'sku' y 'price' (ahora convertido a float)
    manager.register_context(
        "product_update",
        [
            NormalizeAmountOperation(field_name="price"),
            ContextualFieldValidation(field_name="sku", pattern=r".+"),
            # Ahora que 'price' debería ser float, validamos que sea ≥ 0
            ContextualFieldValidation(field_name="price", pattern=r"^[0-9]+(?:\.[0-9]+)?$"),
        ]
    )


    # 6.4 Ejemplo de registros a procesar
    sample_records = [
        # 6.4.1 Registro válido de tipo 'order_event'
        {
            "__type__": "order_event",
            "order_id": "ORD789",
            "customer_name": "luis vargas",
            "amount": "123.45",
            "timestamp": "2023-10-26T14:00:00Z"
        },
        # 6.4.2 Registro inválido de tipo 'order_event' (formato de timestamp incorrecto)
        {
            "__type__": "order_event",
            "order_id": "ORD999",
            "customer_name": "ana pérez",
            "amount": "1.234,56 €",
            "timestamp": "26-10-2023 14:00:00"  # no cumple ISO 8601
        },
        # 6.4.3 Registro válido de tipo 'product_update'
        {
            "__type__": "product_update",
            "sku": "SKU_P01",
            "price": "59,99 EUR"
        },
        # 6.4.4 Registro inválido de tipo 'product_update' (price no es monetario o es negativo)
        {
            "__type__": "product_update",
            "sku": "SKU_XYZ",
            "price": "XYZ99.99"
        },
        # 6.4.5 Registro sin '__type__'
        {
            "order_id": "ORD_NO_TYPE",
            "amount": "10.00"
        }
    ]


    # 6.5 Procesar el flujo de registros y mostrar resultado
    for rec, problems in manager.process_stream(sample_records):
        print("\n--- Registro procesado: ---")
        for k, v in rec.items():
            print(f"  {k!r}: {v!r}")
        if problems:
            print("  >> Problemas encontrados:")
            for msg in problems:
                print(f"     - {msg}")
        else:
            print("  >> Registro válido sin problemas.")



--- Registro procesado: ---
  '__type__': 'order_event'
  'order_id': 'ORD789'
  'customer_name': 'luis vargas'
  'amount': 123.45
  'timestamp': '2023-10-26T14:00:00Z'
  '_valid': True
  '_errors': []
  >> Registro válido sin problemas.

--- Registro procesado: ---
  '__type__': 'order_event'
  'order_id': 'ORD999'
  'customer_name': 'ana pérez'
  'amount': 1234.56
  'timestamp': '26-10-2023 14:00:00'
  '_valid': False
  '_errors': ["[ContextualFieldValidation] El campo 'timestamp' (valor='26-10-2023 14:00:00') no coincide con el patrón /^\\d{4}-\\d{2}-\\d{2}T\\d{2}:\\d{2}:\\d{2}Z$/."]
  >> Problemas encontrados:
     - [ContextualFieldValidation] El campo 'timestamp' (valor='26-10-2023 14:00:00') no coincide con el patrón /^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$/.

--- Registro procesado: ---
  '__type__': 'product_update'
  'sku': 'SKU_P01'
  'price': 59.99
  '_valid': True
  '_errors': []
  >> Registro válido sin problemas.

--- Registro procesado: ---
  '__type__': 'product_update

# Justificacion

**Uso de una lista de funciones puras por operación:**

**Ventaja:** Las funciones puras son ideales para operaciones específicas y determinísticas. Esto significa que una vez que se define una operación para un campo (por ejemplo, NormalizeAmountOperation), su comportamiento será predecible y no tendrá efectos secundarios. Esto simplifica la depuración y el mantenimiento del sistema.

**Desventaja:** Sin embargo, este enfoque puede ser limitado cuando los registros son muy dinámicos o cuando se requiere una lógica más compleja que no se puede expresar fácilmente como una función pura. Además, al manejar múltiples operaciones de diferentes tipos, el mantenimiento de funciones individuales puede volverse difícil cuando hay cambios en los requisitos.

**Uso de eval() o manipulación de cadenas:**

**Ventaja:** Usar eval() permite la ejecución dinámica de código y puede ser útil para crear operaciones personalizadas sobre los datos de los registros sin tener que escribir manualmente cada operación. Este enfoque es flexible y puede ser adaptado a cambios de lógica de negocio o nuevas operaciones.

**Desventaja:** La principal desventaja de eval() es la seguridad. Si se usan datos de entrada no controlados, eval() podría ejecutar código malicioso. Además, la manipulación de cadenas puede resultar propensa a errores si no se validan y sanitizan adecuadamente los datos.

**Para la operación NormalizeAmountOperation: Flexibilidad para cualquier campo numérico:**

**Pregunta: ¿Cómo asegurarse de que la operación NormalizeAmountOperation sea flexible para cualquier campo numérico, no solo amount o price?**

**Solución:** Para lograr flexibilidad, podemos permitir que NormalizeAmountOperation acepte cualquier nombre de campo y pueda ser reutilizada en cualquier tipo de registro que contenga un campo numérico. De esta manera, no necesitamos definir operaciones diferentes para cada campo, sino que la operación se adapta a los registros según el campo especificado en la configuración.

**Manejo de campos que no existen en el registro:**

**Pregunta: ¿Cómo manejaría el caso donde el campo a normalizar no existe en el registro?**

**Solución:** Si el campo no existe en el registro, la operación debe devolver None o una advertencia. También puede agregar un mecanismo de validación que permita comprobar la existencia del campo antes de intentar la conversión.

**Implementación Sugerida:** Para mejorar la flexibilidad de la operación y permitirla para cualquier campo numérico, podemos hacer ajustes al código para hacerlo más dinámico

In [9]:
records = [
    # --- Casos válidos ---
    {
      "__type__": "order_event",
      "order_id": "ORD789",
      "customer_name": "Luis Vargas",
      "amount": "123,45 EUR",
      "timestamp": "2024-10-26T14:00:00Z"
    },
    # --- Casos no válidos ---
    {
      "__type__": "order_event",
      "order_id": "ORD100",
      "customer_name": "Bob el Constructor",
      "amount": "no_es_un_numero",
      "timestamp": "2024-13-01T25:61:00Z"
    },
    {
      "__type__": "product_update",
      "product_sku": "SKU_P002",
      "price": None,
      "is_active": "False"
    },
    {
      "__type__": "product_update",
      "product_sku": "SKU_P003",
      "price": "25.00",
      # "is_active" faltante
    }
]


In [10]:
from typing import Dict, List, Any, Iterable, Tuple
import re, logging
from datetime import datetime
import abc

# Definición de Operaciones y Manager

logging.basicConfig(level=logging.WARNING, format="%(asctime)s [%(levelname)s] %(name)s: %(message)s")

class Operation(abc.ABC):
    def __init__(self, field_name: str):
        self.field_name = field_name
    @abc.abstractmethod
    def apply(self, record: dict) -> None:
        pass

class NormalizeAmountOperation(Operation):
    def __init__(self, field_name: str):
        super().__init__(field_name)
        self.logger = logging.getLogger(self.__class__.__name__)
    def apply(self, record: dict) -> None:
        if self.field_name not in record:
            self.logger.warning(f"[NormalizeAmountOperation] Campo '{self.field_name}' no existe; omitiendo.")
            return
        raw_str = str(record[self.field_name])
        cleaned = re.sub(r"[^\d\.,\-]", "", raw_str)
        if "," in cleaned:
            if "." in cleaned:
                without_dots = cleaned.replace(".", "")
                normalized = without_dots.replace(",", ".")
            else:
                normalized = cleaned.replace(",", ".")
        else:
            normalized = cleaned
        try:
            record[self.field_name] = float(normalized)
        except Exception:
            self.logger.warning(f"[NormalizeAmountOperation] No se pudo convertir '{raw_str}' -> '{normalized}'")

class ContextualFieldValidation(Operation):
    def __init__(self, field_name: str, pattern: str):
        super().__init__(field_name)
        self.pattern = re.compile(pattern)
    def apply(self, record: dict) -> None:
        if self.field_name not in record:
            raise ValueError(f"[ContextualFieldValidation] Falta campo '{self.field_name}'.")
        raw = str(record[self.field_name])
        if not self.pattern.fullmatch(raw):
            raise ValueError(f"[ContextualFieldValidation] Campo '{self.field_name}'='{raw}' no coincide con /{self.pattern.pattern}/.")

class RecordContextManager:
    def __init__(self):
        self._contexts: Dict[str, List[Operation]] = {}
    def register_context(self, tipo: str, operations: List[Operation]) -> None:
        self._contexts[tipo] = operations
    def process_stream(self, record_iterator: Iterable[Dict[str, Any]]) -> Iterable[Tuple[Dict[str, Any], List[str]]]:
        for original in record_iterator:
            record = original.copy()
            errors: List[str] = []
            tipo = record.get("__type__")
            if tipo is None:
                errors.append("Falta campo '__type__'; no se puede aplicar contexto.")
                record["_valid"] = False
                record["_errors"] = errors
                yield record, errors
                continue
            ops = self._contexts.get(tipo, [])
            for op in ops:
                try:
                    op.apply(record)
                except ValueError as ve:
                    errors.append(str(ve))
            record["_valid"] = (len(errors) == 0)
            record["_errors"] = errors
            yield record, errors

# -Configuración de Contextos

manager = RecordContextManager()

manager.register_context(
    "order_event",
    [
        NormalizeAmountOperation("amount"),
        ContextualFieldValidation("order_id",        r".+"),
        ContextualFieldValidation("customer_name",   r".+"),
        ContextualFieldValidation("timestamp",       r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")
    ]
)

manager.register_context(
    "product_update",
    [
        # Primero convertimos el campo "price" a float, si existe
        NormalizeAmountOperation("price"),
        # Luego validamos que "product_sku" (nota: en los datos originales usaron "product_sku" en lugar de "sku")
        ContextualFieldValidation("product_sku", r".+"),
        # Validamos que, tras normalizar, "price" sea un número ≥ 0
        ContextualFieldValidation("price",       r"^[0-9]+(?:\.[0-9]+)?$")
    ]
)

# Flujos de Prueba

records = [
    {
      "__type__": "order_event",
      "order_id": "ORD789",
      "customer_name": "Luis Vargas",
      "amount": "123,45 EUR",
      "timestamp": "2024-10-26T14:00:00Z"
    },
    {
      "__type__": "order_event",
      "order_id": "ORD100",
      "customer_name": "Bob el Constructor",
      "amount": "no_es_un_numero",
      "timestamp": "2024-13-01T25:61:00Z"
    },
    {
      "__type__": "product_update",
      "product_sku": "SKU_P002",
      "price": None,
      "is_active": "False"
    },
    {
      "__type__": "product_update",
      "product_sku": "SKU_P003",
      "price": "25.00",
      # "is_active"
    }
]

for rec, problems in manager.process_stream(records):
    print("------ Registro Procesado ------")
    for k, v in rec.items():
        print(f"  {k!r}: {v!r}")
    if problems:
        print("  >>> Errores detectados:")
        for msg in problems:
            print(f"      * {msg}")
    else:
        print("  >>> Sin errores (válido).")
    print()


------ Registro Procesado ------
  '__type__': 'order_event'
  'order_id': 'ORD789'
  'customer_name': 'Luis Vargas'
  'amount': 123.45
  'timestamp': '2024-10-26T14:00:00Z'
  '_valid': True
  '_errors': []
  >>> Sin errores (válido).

------ Registro Procesado ------
  '__type__': 'order_event'
  'order_id': 'ORD100'
  'customer_name': 'Bob el Constructor'
  'amount': 'no_es_un_numero'
  'timestamp': '2024-13-01T25:61:00Z'
  '_valid': True
  '_errors': []
  >>> Sin errores (válido).

------ Registro Procesado ------
  '__type__': 'product_update'
  'product_sku': 'SKU_P002'
  'price': None
  'is_active': 'False'
  '_valid': False
  '_errors': ["[ContextualFieldValidation] Campo 'price'='None' no coincide con /^[0-9]+(?:\\.[0-9]+)?$/."]
  >>> Errores detectados:
      * [ContextualFieldValidation] Campo 'price'='None' no coincide con /^[0-9]+(?:\.[0-9]+)?$/.

------ Registro Procesado ------
  '__type__': 'product_update'
  'product_sku': 'SKU_P003'
  'price': 25.0
  '_valid': True
  '